In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import time

In [2]:
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from scipy import stats
#from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score


IMPORT TRAINING DATA

In [3]:
data=pd.read_csv(r'trainingdata.csv')
print(data.shape)

(100000, 132)


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 132 entries, row ID to loss
dtypes: float64(15), int64(1), object(116)
memory usage: 100.7+ MB


In [5]:
data.head()

,row ID,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,loss
0,1,LB,O,A,S,AS,C,BC,BU,G,...,A,D,B,C,E,A,C,T,B,3325.946771
1,2,DP,O,A,BM,AV,A,CQ,BI,K,...,A,D,D,C,E,E,D,T,L,2212.393559
2,3,GK,I,A,AF,C,A,DK,AB,A,...,A,D,D,C,E,E,A,D,L,3903.026647
3,4,DJ,O,A,AE,N,C,CS,BI,K,...,A,D,D,C,E,E,D,T,I,2045.297525
4,5,CK,K,A,BM,Y,C,C,H,B,...,H,D,B,D,E,E,A,P,F,3479.835087


## *This is where you will do some preprocessing*

DEFINE X/y

In [6]:
target = "loss"

In [7]:
cleaned_df = data.copy()

In [8]:
X=cleaned_df.drop(columns=[target])
y=cleaned_df[target]
y

0        3325.946771
1        2212.393559
2        3903.026647
3        2045.297525
4        3479.835087
            ...     
99995    4660.717696
99996    3352.398155
99997    2952.048273
99998    1769.994777
99999    2077.743445
Name: loss, Length: 100000, dtype: float64

LABEL ENCODE CATEGORICAL VARIABLES
*(You can also use one-hot encoding)*

In [9]:
#cat_cols = X.select_dtypes(include=['object']).columns
#le = LabelEncoder()
#X = X[cat_cols].apply(lambda x: le.fit_transform(x))

X = pd.get_dummies(X)

In [10]:
X

,row ID,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,cat100_F,cat100_G,cat100_H,cat100_I,cat100_J,cat100_K,cat100_L,cat100_M,cat100_N,cat100_O
0,1,0.569745,0.594646,0.822493,0.714843,0.310061,0.718367,0.335060,0.30260,0.67135,...,0,0,0,0,0,0,0,0,0,0
1,2,0.338312,0.366307,0.611431,0.304496,0.885834,0.438917,0.436585,0.60087,0.35127,...,0,0,0,0,0,0,1,0,0,0
2,3,0.381398,0.373424,0.195709,0.774425,0.397069,0.289648,0.315545,0.27320,0.26076,...,0,0,0,0,0,0,1,0,0,0
3,4,0.327915,0.321570,0.605077,0.602642,0.422268,0.440945,0.391128,0.31796,0.32128,...,0,0,0,1,0,0,0,0,0,0
4,5,0.204687,0.202213,0.246011,0.432606,0.704268,0.178193,0.247408,0.24564,0.22089,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,0.888438,0.879347,0.846403,0.793236,0.281143,0.923434,0.954830,0.38885,0.81945,...,0,0,0,0,0,0,0,0,0,1
99996,99997,0.644013,0.785706,0.862949,0.721447,0.534484,0.874303,0.675139,0.64577,0.64296,...,1,0,0,0,0,0,0,0,0,0
99997,99998,0.531161,0.519456,0.304350,0.815292,0.281143,0.443265,0.618755,0.36636,0.52938,...,0,0,0,1,0,0,0,0,0,0
99998,99999,0.291268,0.286079,0.287682,0.285859,0.836443,0.389874,0.398557,0.53642,0.42700,...,0,0,0,0,0,0,0,0,0,0


TRAIN/VALIDATION SPLIT 
*(also called train/test split)*

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3)

FIT MULTIPLE MODELS TO DECIDE THE BEST ONE 
(*criterion: Least RMSE*)

In [12]:
def fitting_models():
    lr=LinearRegression()
    ls = Lasso(alpha=2)
    rg = Ridge(alpha=8)
    gbr = GradientBoostingRegressor(random_state = 69, n_estimators = 1000)
    #br = BaggingRegressor(n_estimators = 26, random_state = 5)
    
    rgs = [('Linear Regression', lr),
        ('Lasso', ls),
        ('Ridge', rg),
      #  ('Bagging Regressor', br)
        ('GBR', gbr)
              
  ]
    for name,lr in rgs:
        lr.fit(X_train,y_train)
        pred = lr.predict(X_test)
        score = format(r2_score(y_test,pred), '.4f')
        rmse = np.sqrt(mean_squared_error(y_test, pred))
        print("{} : {} , {}".format(name,score,rmse))

In [16]:
def fitting_models_CV2():
    cv = RepeatedKFold(n_splits=2, n_repeats=1)#, random_state=1)
    
    lr=LinearRegression()
    ls = Lasso(alpha=10.5)
    rg = Ridge(alpha=1.5)
    gbr = GradientBoostingRegressor(random_state = 420, n_estimators = 200) # we can specify loss as huber as one of the parameters.
    knr = KNeighborsRegressor(n_neighbors=5)
    pipe_knr = Pipeline([("scaler", MinMaxScaler()), 
                         ("knr", KNeighborsRegressor(n_neighbors=5))])
    rgs = [#('Linear Regression', lr),
        #('Lasso Regression', ls),
        ('Gradient Boosting', gbr),   
        #('KNearest Neighbor',knr),
        #('Scaled KNearest',pipe_knr)       
    ]
    for name,rg in rgs:
        start = time.perf_counter()
        scores = cross_val_score(rg, X, y, cv=cv, scoring = 'neg_root_mean_squared_error') 
        end = time.perf_counter()        
        score = format(np.mean(scores), '.4f')
        duration = format((end-start),'.4f')
        print("{} : {} - {}".format(name,score,duration))

In [ ]:
fitting_models()

In [17]:
fitting_models_CV2()

ValueError: too many values to unpack (expected 2)

In [25]:
rg=Ridge(alpha = 8).fit(X_train, y_train)

In [64]:
gbr = GradientBoostingRegressor(random_state = 420, n_estimators = 200).fit(X_train,y_train)

In [78]:
Ngbr = GradientBoostingRegressor(random_state = 69, n_estimators = 200).fit(X_train,y_train)

In [ ]:
ttgbr = TransformedTargetRegressor(regressor = GradientBoostingRegressor(random_state = 420, n_estimators = 200), transformer = MinMaxScaler()).fit(X, y)


IMPORT TESTING DATA FOR PREDICTIONS

In [31]:
test_data = pd.read_csv(r'testingdata.csv')
print(test_data.shape)

(2001, 131)


TRANSFORM THE TEST DATA SANE WAY AS YOU TRANSFORMED TRAIN DATA

In [32]:
cat_cols = test_data.select_dtypes(include=['object']).columns
le = LabelEncoder()
test_data = test_data[cat_cols].apply(lambda x: le.fit_transform(x))
test_data

,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,...,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100
0,31,6,0,2,29,0,35,10,1,4,...,0,0,3,3,2,2,1,0,8,11
1,95,10,0,26,8,2,23,10,10,8,...,0,0,3,3,1,2,1,3,11,8
2,35,10,0,26,14,4,23,10,10,9,...,0,0,3,3,1,2,3,3,11,8
3,75,6,0,35,16,1,38,10,1,4,...,0,0,3,1,2,2,1,0,8,14
4,129,10,0,36,18,1,38,10,6,6,...,0,0,2,1,1,2,0,2,11,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,87,9,0,1,38,0,20,10,10,7,...,6,3,3,3,1,2,0,3,11,6
1997,31,6,0,26,34,0,25,10,1,4,...,2,0,2,3,2,2,1,0,8,11
1998,78,9,0,3,38,1,14,10,10,9,...,0,0,3,3,1,2,1,3,11,6
1999,82,9,0,10,6,0,20,10,4,8,...,0,0,3,3,1,2,1,3,11,8


PICK THE BEST MODEL FOR PREDICTING ON TEST DATA

In [42]:
predictions = ttgbr.predict(test_data)
print(predictions)

[4684.51076342 4690.20023827 2784.99761493 ... 2728.84158219 3643.24665181
 3126.22059812]


FORMAT IT ACCORDING TO SAMPLE.CSV (*shared on kaggle*)

In [43]:
predictions_df=pd.DataFrame(predictions, columns=['loss'])
predictions_df

,loss
0,4684.510763
1,4690.200238
2,2784.997615
3,2267.079532
4,4303.021618
...,...
1996,5755.060088
1997,2566.494720
1998,2728.841582
1999,3643.246652


In [44]:
predictions_df.insert(0, 'row ID', range(100001, 102002))
predictions_df

,row ID,loss
0,100001,4684.510763
1,100002,4690.200238
2,100003,2784.997615
3,100004,2267.079532
4,100005,4303.021618
...,...,...
1996,101997,5755.060088
1997,101998,2566.494720
1998,101999,2728.841582
1999,102000,3643.246652


EXPORT PREDICTION DATAFRAME IN CSV

In [45]:
predictions_df.to_csv('predictionsTTOHE.csv', sep=',', index=False)